In [27]:
"""
    Inspired from toma - https://github.com/BlackHC/toma
"""

import gc
import torch

def batch(func,*,batch_size = None,):
    gc_cuda()

    #
#     if batch_size is None and 'batch_size' in kwargs:
#         batch_size = kwargs['batch_size']

    while True:
        try:
            print(f"Im in the decorator - {func.__name__}")
            return func(batch_size)
        except RuntimeError as exception:
            if batch_size > 1 and should_reduce_batch_size(exception):
                batch_size //= 2
                gc_cuda()
            else:
                raise

def gc_cuda():
    """Gargage collect Torch (CUDA) memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def should_reduce_batch_size(exception):
    return is_cuda_out_of_memory(exception) or is_cudnn_snafu(exception) or is_out_of_cpu_memory(exception)

def is_cuda_out_of_memory(exception):
    return (
        isinstance(exception, RuntimeError) and len(exception.args) == 1 and "CUDA out of memory." in exception.args[0]
    )

def is_cudnn_snafu(exception):
    # For/because of https://github.com/pytorch/pytorch/issues/4107
    return (
        isinstance(exception, RuntimeError)
        and len(exception.args) == 1
        and "cuDNN error: CUDNN_STATUS_NOT_SUPPORTED." in exception.args[0]
    )

def is_out_of_cpu_memory(exception):
    return (
        isinstance(exception, RuntimeError)
        and len(exception.args) == 1
        and "DefaultCPUAllocator: can't allocate memory" in exception.args[0]
    )

In [1]:
def batch(func):
    """Run inference by deciding appropriate batch size on input
    """
    def inner_wrapper(*args, batch_size, **kwargs):
        while True:
            try:
                print(f"func - {args}, kwargs - {kwargs}, batch_size - {batch_size}")
                res = func(*args,batch_size = batch_size, **kwargs)
                return res
            except RuntimeError as exception:
                raise
                if batch_size > 1 and should_reduce_batch_size(exception):
                    batch_size //= 2
                    gc_cuda()
                else:
                    raise
    
    return inner_wrapper

@batch
def foo(a,b, batch_size = 8):
    """
    a, b - batch of data
    
    default batch size - 8 will try to reduce by half if it doesn't fit in memory
    """
    print(f"in function foo result - {a+b}, batch_size - {batch_size}")
    
foo(2, b = 4, batch_size = 4)

func - (2,), kwargs - {'b': 4}, batch_size - 4
in function foo result - 6, batch_size - 4


In [29]:
@batch
def foo(batch_size, a = 1, b = 2):
    print(f"I'm in function foo - {batch_size}")
    
foo(batch_size = None)

Im in the decorator - foo
I'm in function foo - None


TypeError: 'NoneType' object is not callable